In [1]:
import getpass
import os

os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your GROQ API key: ")

In [2]:
!pip install -qU langchain-groq

In [3]:
!pip install -qU langchain-huggingface

In [4]:
!pip install -qU langchain-chroma

In [10]:
!pip install -qU langchain_community jq 

In [16]:
!pip install -qU langchain_community unstructured

In [17]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.7,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

In [18]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [19]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="C:\\Users\\bhara\\OneDrive\\Desktop\\Pro_1",  # Where to save data locally, remove if not necessary
)

In [ ]:
import glob
from langchain_community.document_loaders import UnstructuredMarkdownLoader, JSONLoader

# --- Load all Markdown files ---
markdown_files = glob.glob("./markdown_files/*.md")
all_documents = []

for file_path in markdown_files:
    loader = UnstructuredMarkdownLoader(
        file_path=file_path,
        metadata={"source": file_path},  # Include filename as metadata
        mode="single",
        strategy="fast",
    )
    documents = loader.load()
    all_documents.extend(documents)

json_loader = JSONLoader(
    file_path="discourse_posts.json",
    jq_schema=".[].content",  # Adjust jq path to match your JSON structure
    text_content=False,
)

json_documents = json_loader.load()
all_documents.extend(json_documents)

# Now, `all_documents` contains both Markdown and JSON loaded data


882

In [24]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(all_documents)

In [25]:
_ = vector_store.add_documents(documents=all_splits)

In [26]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

c:\Users\bhara\anaconda3\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [27]:
# Define application steps
from langgraph.graph import START, StateGraph
from typing import List, TypedDict
from langchain_core.documents import Document

# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [34]:
response = graph.invoke({"question": "The question asks to use gpt-3.5-turbo-0125 model but the ai-proxy provided by Anand sir only supports gpt-4o-mini. So should we just use gpt-4o-mini or use the OpenAI API for gpt3.5 turbo?"})
print(response["answer"])

You should use the OpenAI API directly for this question, as it requires the 'gpt-3.5-turbo-0125' model which is not supported by the AI proxy provided by Anand sir. Using the OpenAI API is the best option for this specific question.
